# Bounding Box Visualization of Leaf Diseases (YOLOv8 + .keras CNN)

In [1]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
# Load your trained CNN model (.keras format)
cnn_model = load_model("/path/to/trained_plant_disease_model.keras")

In [3]:
# Define preprocessing (adjust if needed based on your model's training)
def preprocess(image):
    image = cv2.resize(image, (128, 128))  # Match CNN input size
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

In [4]:
# Define label decoding (customize based on your model's output)
labels =['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']

In [5]:
def decode_prediction(pred):
    class_idx = np.argmax(pred)
    return labels[class_idx], float(pred[0][class_idx])

In [6]:
yolo_model = YOLO("yolov8n.pt")

In [ ]:
# Read input image with multiple leaves
image_path = "/path/to/uav_image_for_testing.jpg"
image = cv2.imread(image_path)

In [23]:
# Run object detection
results = yolo_model(image)


0: 448x640 1 cup, 3 cakes, 235.3ms
Speed: 5.3ms preprocess, 235.3ms inference, 11.6ms postprocess per image at shape (1, 3, 448, 640)


In [24]:
# Make directory for crops if needed
os.makedirs("leaf_crops", exist_ok=True)

In [25]:
# For each detection box
for i, box in enumerate(results[0].boxes.xyxy):
    x1, y1, x2, y2 = map(int, box)
    leaf_crop = image[y1:y2, x1:x2]

    # Save optional crop
    crop_path = f"leaf_crops/leaf_{i+1}.jpg"
    cv2.imwrite(crop_path, leaf_crop)

    # Preprocess and predict with CNN
    input_img = preprocess(leaf_crop)
    prediction = cnn_model.predict(input_img)
    label, confidence = decode_prediction(prediction)

    # Draw result on image
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, f"{label} ({confidence*100:.1f}%)", (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

1/1 [==============================] - 0s 45ms/step


In [26]:
# Save annotated image
cv2.imwrite("detected_and_classified.jpg", image)

True